Querying file and Extracting data 

In [0]:
SELECT * ,_metadata.file_path as path FROM read_files('/Volumes/gizmobox/landing/operational_data/customers/*.json')

In [0]:
create or replace view gizmobox.bronze.v_customers
as SELECT * ,_metadata.file_path as path FROM read_files('/Volumes/gizmobox/landing/operational_data/customers/')

In [0]:
select current_catalog();
USE catalog gizmobox;
use schema bronze;
select * from v_customers